In [1]:
import mbuild as mb
from mbuild.lib.recipes import TiledCompound
import parmed as pmd
import numpy as np
import openmm
import warnings
from random import sample
from copy import deepcopy
import openbabel as ob
from copy import deepcopy
# from slab_builder import *
warnings.simplefilter("ignore")

/home/robert/miniconda3/envs/dev/lib/python3.11/site-packages/mdtraj/formats/__init__.py:6: DeprecationWarning: 'xdrlib' is deprecated and slated for removal in Python 3.13
  from .xtc import XTCTrajectoryFile
/home/robert/miniconda3/envs/dev/lib/python3.11/site-packages/lark/utils.py:163: DeprecationWarning: module 'sre_parse' is deprecated
  import sre_parse
/home/robert/miniconda3/envs/dev/lib/python3.11/site-packages/lark/utils.py:164: DeprecationWarning: module 'sre_constants' is deprecated
  import sre_constants


In [2]:
import ase
from ase import Atoms
from ase.build import surface
from ase.visualize import view
from ase.io import write

In [4]:
from ase.build import fcc111
slab = fcc111('Al', size=(2,2,3), vacuum=10.0)
view(slab)

<Popen: returncode: None args: ['/home/robert/miniconda3/envs/dev/bin/python...>

In [5]:
from ase.build import fcc110
slab = fcc110('Al', size=(2,2,3), vacuum=10.0)
view(slab)

<Popen: returncode: None args: ['/home/robert/miniconda3/envs/dev/bin/python...>

In [6]:
a = 5.39
Pt3Rh = Atoms('ZnS',
              scaled_positions=[(0, 0, 0),
                                (0.25, 0.75, 0.75)],
              cell=[a, a, a],
              pbc=True)
s3 = surface(Pt3Rh, (1, 1, 0), 6)
s3.center(vacuum=0, axis=2)

Pt3Rh.set_chemical_symbols('SZn')
s4 = surface(Pt3Rh, (1, 1, 0), 9)
s4.center(vacuum=0, axis=2)

In [7]:
big_110 = ase.build.make_supercell(s3,([6,0,0],[0,6,0],[0,0,1]))

view(big_110)

<Popen: returncode: None args: ['/home/robert/miniconda3/envs/dev/bin/python...>

In [46]:
write("ZnS_slab_110_6x6x6.xyz", big_110)

In [11]:
write("ZnS_slab_110_6x6x6.lammps", big_110, "lammps-data")

In [25]:
view(s4, viewer="nglview")

In [8]:
write?

Signature:
write(
    filename: Union[str, pathlib.PurePath, IO],
    images: Union[ase.atoms.Atoms, Sequence[ase.atoms.Atoms]],
    format: str = None,
    parallel: bool = True,
    append: bool = False,
    **kwargs: dict,
) -> None
Docstring:
Write Atoms object(s) to file.

filename: str or file
    Name of the file to write to or a file descriptor.  The name '-'
    means standard output.
images: Atoms object or list of Atoms objects
    A single Atoms object or a list of Atoms objects.
format: str
    Used to specify the file-format.  If not given, the
    file-format will be taken from suffix of the filename.
parallel: bool
    Default is to write on master only.  Use parallel=False to write
    from all slaves.
append: bool
    Default is to open files in 'w' or 'wb' mode, overwriting
    existing files.  In some cases opening the file in 'a' or 'ab'
    mode (appending) is useful,
    e.g. writing trajectories or saving multiple Atoms objects in one file.
    additional keywor